In [37]:
import os
import shutil
import cv2
import pandas as pd
import numpy as np
np.random.seed(42)
import random

DATA_PATH = '/opt/ml/input/cropped_v2.1/train/'
IMG_PATH = '/opt/ml/input/cropped_v2.1/train/images/'
# NEW_IMG_PATH = '/opt/ml/input/cropped_v2/train/images_classified/'
# NEW_IMG_PATH = '/opt/ml/input/cropped_v2.1/train/images_classified_mask/'
# NEW_IMG_PATH = '/opt/ml/input/cropped_v2.1/train/images_classified_gender/'
NEW_IMG_PATH = '/opt/ml/input/cropped_v2.1/train/images_classified_age/'
# train_classified = 'train_classified.csv'
# train_classified = 'train_classified_mask.csv'
# train_classified = 'train_classified_gender.csv'
train_classified = 'train_classified_age.csv'

In [38]:
df = pd.read_csv(DATA_PATH+'train.csv')
df_cls = pd.read_csv(DATA_PATH+train_classified)

In [39]:
if not os.path.isdir(NEW_IMG_PATH):
    os.makedirs(NEW_IMG_PATH)

In [40]:
df

,id,gender,race,age,path
0,000001,female,Asian,45,000001_female_Asian_45
1,000002,female,Asian,52,000002_female_Asian_52
2,000004,male,Asian,54,000004_male_Asian_54
3,000005,female,Asian,58,000005_female_Asian_58
4,000006,female,Asian,59,000006_female_Asian_59
...,...,...,...,...,...
2695,006954,male,Asian,19,006954_male_Asian_19
2696,006955,male,Asian,19,006955_male_Asian_19
2697,006956,male,Asian,19,006956_male_Asian_19
2698,006957,male,Asian,20,006957_male_Asian_20


In [41]:
df_cls

,id,gender,race,age,path,img_path,name,mask,target
0,000001,female,Asian,45,000001_female_Asian_45,000001_female_Asian_45/incorrect_mask.jpg,incorrect_mask.jpg,Incorrect,1
1,000001,female,Asian,45,000001_female_Asian_45,000001_female_Asian_45/mask1.jpg,mask1.jpg,Wear,1
2,000001,female,Asian,45,000001_female_Asian_45,000001_female_Asian_45/mask2.jpg,mask2.jpg,Wear,1
3,000001,female,Asian,45,000001_female_Asian_45,000001_female_Asian_45/mask3.jpg,mask3.jpg,Wear,1
4,000001,female,Asian,45,000001_female_Asian_45,000001_female_Asian_45/mask4.jpg,mask4.jpg,Wear,1
...,...,...,...,...,...,...,...,...,...
18895,006959,male,Asian,19,006959_male_Asian_19,006959_male_Asian_19/mask2.jpg,mask2.jpg,Wear,0
18896,006959,male,Asian,19,006959_male_Asian_19,006959_male_Asian_19/mask3.jpg,mask3.jpg,Wear,0
18897,006959,male,Asian,19,006959_male_Asian_19,006959_male_Asian_19/mask4.jpg,mask4.jpg,Wear,0
18898,006959,male,Asian,19,006959_male_Asian_19,006959_male_Asian_19/mask5.jpg,mask5.jpg,Wear,0


In [42]:
df_cls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18900 entries, 0 to 18899
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        18900 non-null  object
 1   gender    18900 non-null  object
 2   race      18900 non-null  object
 3   age       18900 non-null  int64 
 4   path      18900 non-null  object
 5   img_path  18900 non-null  object
 6   name      18900 non-null  object
 7   mask      18900 non-null  object
 8   target    18900 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 1.3+ MB


In [43]:
df_cls['target'].unique()

array([1, 0, 2])

In [44]:
# for i in range(len(df_cls['target'].unique())):
for i in df_cls['target'].unique():
    if not os.path.isdir(NEW_IMG_PATH+str(i)):
        os.makedirs(NEW_IMG_PATH+str(i))

In [45]:
path_list = sorted([p for p in os.listdir(IMG_PATH) if '._' not in p])
path_list


['000001_female_Asian_45',
 '000002_female_Asian_52',
 '000004_male_Asian_54',
 '000005_female_Asian_58',
 '000006_female_Asian_59',
 '000007_female_Asian_58',
 '000008_female_Asian_58',
 '000009_female_Asian_56',
 '000010_female_Asian_58',
 '000012_male_Asian_57',
 '000014_female_Asian_58',
 '000015_male_Asian_55',
 '000016_female_Asian_58',
 '000018_female_Asian_59',
 '000019_female_Asian_57',
 '000020_female_Asian_50',
 '000021_female_Asian_54',
 '000022_female_Asian_58',
 '000023_female_Asian_55',
 '000024_female_Asian_58',
 '000025_female_Asian_49',
 '000026_female_Asian_56',
 '000028_female_Asian_58',
 '000029_female_Asian_58',
 '000030_female_Asian_55',
 '000031_female_Asian_54',
 '000033_male_Asian_57',
 '000034_female_Asian_55',
 '000036_female_Asian_58',
 '000037_female_Asian_58',
 '000038_female_Asian_58',
 '000039_female_Asian_57',
 '000040_female_Asian_57',
 '000041_female_Asian_58',
 '000043_male_Asian_56',
 '000044_female_Asian_58',
 '000045_female_Asian_56',
 '000046_ma

In [46]:
df_cls.iloc[1, -4]

'000001_female_Asian_45/mask1.jpg'

In [47]:
# for p in path_list:
#     idx_list = df_cls.index[df_cls['path'] == p].tolist()
#     for i in idx_list:
#         label = str(df_cls.iloc[i, -1])
#         img_path = df_cls.iloc[i, -4]
#         path, file_name = img_path.split('/')
#         print(IMG_PATH+img_path, NEW_IMG_PATH+label+'/'+path + '_' + file_name)

In [48]:
def filp_img(target_path):
    origin = cv2.imread(target_path)
    img = origin.copy()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # 좌우반전
    img = cv2.flip(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    # cv2.imwrite(save_path, img)
    return img

def rotate_img(target_path, rotate_angle=5, zoomit=1.1):
    '''
    Recommended
    rotate_angle 5, zoomit=1.1
    rotate_angle -5, zoomit=1.1
    rotate_angle 10, zoomit=1.2
    rotate_angle -10, zoomit=1.2
    '''
    origin = cv2.imread(target_path)
    img = origin.copy()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    w, h, c = img.shape
    cross_line = int((w*w + h*h)**0.5)
    rotate_point = int(w/2), int(h/2)
    rotate_option = cv2.getRotationMatrix2D(rotate_point, rotate_angle, zoomit)
    img = cv2.warpAffine(img, rotate_option, (h,w))
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    # cv2.imwrite(save_path, img)
    return img

def contrast_img(target_path, alpha=2):
    origin = cv2.imread(target_path)
    img = origin.copy()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.convertScaleAbs(img, alpha=alpha)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return img

def brightness_img(target_path, brightness=30):
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    origin = cv2.imread(target_path)
    img = origin.copy()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img + brightness
    img = np.clip(img, 10, 255)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return img

def gaussianBlur_img(target_path, val=5):
    origin = cv2.imread(target_path)
    img = origin.copy()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # 필터가 작을수록 선명하다
    img = cv2.GaussianBlur(img, (val,val), 0)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return img

def averaging_img(target_path, val=2):
    origin = cv2.imread(target_path)
    img = origin.copy()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # 필터가 작을수록 선명하다
    img = cv2.blur(img, (val,val))
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return img

In [49]:
# TEST_IMG_PATH = "/opt/ml/input/cropped_v2.1/train/images/"
# sample_img = "000001_female_Asian_45/incorrect_mask.jpg"
# save_path = "./"

# img = filp_img(TEST_IMG_PATH+sample_img)
# cv2.imwrite(save_path+"s_filp.jpg", img)

# img = rotate_img(TEST_IMG_PATH+sample_img)
# cv2.imwrite(save_path+"s_rotate.jpg", img)

# img = contrast_img(TEST_IMG_PATH+sample_img)
# cv2.imwrite(save_path+"s_contrast.jpg", img)

# img = brightness_img(TEST_IMG_PATH+sample_img)
# cv2.imwrite(save_path+"s_brightness.jpg", img)

# img = gaussianBlur_img(TEST_IMG_PATH+sample_img)
# cv2.imwrite(save_path+"s_gaussianBlur.jpg", img)

# img = averaging_img(TEST_IMG_PATH+sample_img)
# cv2.imwrite(save_path+"s_averaging.jpg", img)

In [50]:
cnt = 0

for p in path_list:
    idx_list = df_cls.index[df_cls['path'] == p].tolist()
    for i in idx_list:
        label = str(df_cls.iloc[i, -1])
        img_path = df_cls.iloc[i, -4]
        path, file_name = img_path.split('/')
        
        # Original image
        shutil.copy(IMG_PATH+img_path, f'{NEW_IMG_PATH}{label}/{path}_o_{file_name}')
        
        # filp image
        img = filp_img(IMG_PATH+img_path)
        cv2.imwrite(f'{NEW_IMG_PATH}{label}/{path}_f_{file_name}', img)

        # contrast image
        img = contrast_img(IMG_PATH+img_path)
        cv2.imwrite(f'{NEW_IMG_PATH}{label}/{path}_c_{file_name}', img)

        # brightness image
        img = brightness_img(IMG_PATH+img_path)
        cv2.imwrite(f'{NEW_IMG_PATH}{label}/{path}_b_{file_name}', img)

        # gaussianBlur image
        img = gaussianBlur_img(IMG_PATH+img_path)
        cv2.imwrite(f'{NEW_IMG_PATH}{label}/{path}_g_{file_name}', img)

        # averagingBlur image
        img = averaging_img(IMG_PATH+img_path)
        cv2.imwrite(f'{NEW_IMG_PATH}{label}/{path}_a_{file_name}', img)

        cnt += 1
        ##### 아래는 특전 레이블에 대해 오버샘플링 하도록 하는 코드######
        if label =='6' or label =='12' or label =='2' :
            # rotate image
            img = rotate_img(IMG_PATH+img_path, 5, 1.1)
            cv2.imwrite(f'{NEW_IMG_PATH}{label}/{path}_r5_{file_name}', img)
            img = cv2.flip(img, 1)
            cv2.imwrite(f'{NEW_IMG_PATH}{label}/{path}_fr5_{file_name}', img)

            img = rotate_img(IMG_PATH+img_path, -5, 1.1)
            cv2.imwrite(f'{NEW_IMG_PATH}{label}/{path}_rm5_{file_name}', img)
            img = cv2.flip(img, 1)
            cv2.imwrite(f'{NEW_IMG_PATH}{label}/{path}_frm5_{file_name}', img)

            img = rotate_img(IMG_PATH+img_path, 10, 1.2)
            cv2.imwrite(f'{NEW_IMG_PATH}{label}/{path}_r10_{file_name}', img)
            img = cv2.flip(img, 1)
            cv2.imwrite(f'{NEW_IMG_PATH}{label}/{path}_fr10_{file_name}', img)

            img = rotate_img(IMG_PATH+img_path, -10, 1.2)
            cv2.imwrite(f'{NEW_IMG_PATH}{label}/{path}_rm10_{file_name}', img)
            img = cv2.flip(img, 1)
            cv2.imwrite(f'{NEW_IMG_PATH}{label}/{path}_frm10_{file_name}', img)            

In [51]:
print(cnt)

18900
